# おせちの材料の組合せ最適化問題：量子アニーリングマシン

## 問題が「No solution found」となる原因は様々ですが、以下の点を確認することが解決への手がかりになるかもしれません。

## 制約条件の確認:
制約条件が厳しすぎるために、解が存在しない可能性があります。各制約条件を再確認し、現実的な範囲内で設定されているか確認してください。

## 目的関数の確認:
目的関数が正しく定義されているかを確認してください。特に、コスト関数や制約条件が正しく設定されているかを確認します。

## 変数の範囲の確認:
変数の上限と下限が適切に設定されているか確認してください。例えば、バイナリ変数として定義されている場合、値が0または1の範囲内である必要があります。

## ソルバーの設定:
ソルバーの設定が適切であることを確認します。例えば、FixstarsClientのタイムアウト設定が短すぎると解が見つからないことがあります。タイムアウトを延長する、もしくは他のソルバーのパラメータを調整してみてください。




# ライブラリーのインストール

In [ ]:
!pip install --upgrade amplify

# アクセストークンの設定

In [ ]:
# @title
token = "あなたのアクセストークンを設定" # トークンを入力

# 量子アニーリングマシンのプログラム

In [ ]:
from amplify import BinaryPoly, BinarySymbolGenerator, Solver, sum_poly
from amplify.client import FixstarsClient
from amplify.constraint import equal_to, less_equal, greater_equal
import numpy as np

# Creates a list of the Ingredients
Ingredients = ["Ebi", "Tatukuri", "Kazunoko", "Kuromame", "Tamago"]

# A dictionary of the costs of each of the Ingredients is created
tanka_costs = {
    "Ebi": 3.0,
    "Tatukuri": 5.0,
    "Kazunoko": 4.0,
    "Kuromame": 2.0,
    "Tamago": 1.5,
}

min_unit = {
    "Ebi": 15.0,
    "Tatukuri": 2.0,
    "Kazunoko": 25.0,
    "Kuromame": 3.0,
    "Tamago": 30.0,
}

# A dictionary of the tanpakushitu percent in each of the Ingredients is created
tanpakushitu = {
    "Ebi": 0.18,
    "Tatukuri": 0.7,
    "Kazunoko": 0.29,
    "Kuromame": 0.09,
    "Tamago": 0.1,
}

shibou = {
    "Ebi": 0.01,
    "Tatukuri": 0.05,
    "Kazunoko": 0.02,
    "Kuromame": 0.02,
    "Tamago": 0.08,
}

tansuikabutu = {
    "Ebi": 0.0,
    "Tatukuri": 0.01,
    "Kazunoko": 0.0,
    "Kuromame": 0.54,
    "Tamago": 0.04,
}

enbun = {
    "Ebi": 0.007,
    "Tatukuri": 0.015,
    "Kazunoko": 0.06,
    "Kuromame": 0.002,
    "Tamago": 0.012,
}

# Number of ingredients
n = len(Ingredients)

from amplify import VariableGenerator

gen = VariableGenerator()
ingredient_vars = gen.array("Integer", shape=np.array([n]), bounds=(0, 100))
#print(ingredient_vars)

# Define the cost function
cost_function = sum(
    tanka_costs[Ingredients[i]] * min_unit[Ingredients[i]] * ingredient_vars[i]
    for i in range(n)
)

# Define the constraints
constraints = [
    greater_equal(
        sum(tanpakushitu[Ingredients[i]] * min_unit[Ingredients[i]] * ingredient_vars[i] for i in range(n)),
        100.0,
    ),
    less_equal(
        sum(shibou[Ingredients[i]] * min_unit[Ingredients[i]] * ingredient_vars[i] for i in range(n)),
        50.0,
    ),
    less_equal(
        sum(tansuikabutu[Ingredients[i]] * min_unit[Ingredients[i]] * ingredient_vars[i] for i in range(n)),
        50.0,
    ),
    less_equal(
        sum(enbun[Ingredients[i]] * min_unit[Ingredients[i]] * ingredient_vars[i] for i in range(n)),
        10.0,
    ),
    greater_equal(ingredient_vars[0], 2),  # Ebi
    greater_equal(ingredient_vars[1], 5), # Tatukuri
    greater_equal(ingredient_vars[2], 2),  # Kazunoko
    greater_equal(ingredient_vars[3], 5),  # Kuromame
    greater_equal(ingredient_vars[4], 2),  # Tamago
    less_equal(ingredient_vars[0], 10),    # Ebi
    less_equal(ingredient_vars[1], 30),    # Tatukuri
    less_equal(ingredient_vars[2], 10),    # Kazunoko
    less_equal(ingredient_vars[3], 30),    # Kuromame
    less_equal(ingredient_vars[4], 10),    # Tamago

]
era=5.0
# Combine cost function and constraints
problem = cost_function + era*sum(constraints)

# Set up the client
client = FixstarsClient()
client.token = token  # Set your token here
client.parameters.timeout = 10000  # Set timeout parameter

# Solve the problem
solver = Solver(client)
result = solver.solve(problem)

# Check if a solution was found
if len(result) == 0:
    print("No solution found")

else:
    print()
    solution = result[0].values
    # Use ingredient_vars to access the solution values
    ingredient_values = [solution[var] for var in ingredient_vars]
    for i, ingredient in enumerate(Ingredients):
        # Access the values from the list
        print(f"{ingredient}: {ingredient_values[i]}")

    # Calculate nutritional values
    tanpaku = sum(
        tanpakushitu[ingredient] * min_unit[ingredient] * ingredient_values[i] # Use ingredient_values[i]
        for i, ingredient in enumerate(Ingredients)
    )
    shi = sum(
        shibou[ingredient] * min_unit[ingredient] * ingredient_values[i] # Use ingredient_values[i]
        for i, ingredient in enumerate(Ingredients)
    )
    tan = sum(
        tansuikabutu[ingredient] * min_unit[ingredient] * ingredient_values[i] # Use ingredient_values[i]
        for i, ingredient in enumerate(Ingredients)
    )
    en = sum(
        enbun[ingredient] * min_unit[ingredient] * ingredient_values[i] # Use ingredient_values[i]
        for i, ingredient in enumerate(Ingredients)
    )
    print()
    print(f"tanpakushitu = {tanpaku}")
    print(f"shibou = {shi}")
    print(f"tansuikabutu = {tan}")
    print(f"enbun = {en}")
    print()
    total_cost = sum(
        tanka_costs[Ingredients[i]] * min_unit[Ingredients[i]] * ingredient_values[i] # Use ingredient_values[i]
        for i in range(n)
    ) # Calculate the total cost here
    print(f"Total_Cost_of_Ingredients = {total_cost}") # Print the calculated total cost
    print()
#    print(ingredient_vars)





Ebi: 3.0
Tatukuri: 30.0
Kazunoko: 4.0
Kuromame: 11.0
Tamago: 6.0

tanpakushitu = 100.07
shibou = 20.509999999999998
tansuikabutu = 25.62
enbun = 9.440999999999999

Total_Cost_of_Ingredients = 1171.0



<ipython-input-6-7dfdbe015588>:65: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  ingredient_vars = gen.array("Integer", shape=np.array([n]), bounds=(0, 100))
<ipython-input-6-7dfdbe015588>:76: DeprecationWarning: amplify.constraint.greater_equal is deprecated since amplify v1.0.0 and will no longer support in the near future.
Use amplify.greater_equal instead. Please see the migration guide for details: https://amplify.fixstars.com/docs/amplify/v1/migration.html
  greater_equal(
<ipython-input-6-7dfdbe015588>:80: DeprecationWarning: amplify.constraint.less_equal is deprecated since amplify v1.0.0 and will no longer support in the near future.
Use amplify.less_equal instead. Please see the migration guide for details: https://amplify.fixstars.com/docs/amplify/v1/migration.html
  less_equal(
<ipython-input-6-7dfdb